# Classification including non-exercise data according to Crema <sup>1</sup>


<sup>1</sup> "Characterization of a Wearable System for Automatic Supervision of Fitness Exercises"

In [1]:
import os
import re
import numpy as np
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, TextBox, Button
from sklearn.ensemble import RandomForestClassifier
#import functionsMasterProjectMeinhart as fmpm
#import functionsFeaturesCrema as ffc

import sys
sys.path.append('..')  # in order to import modules from the package 'packageMeinhart'
import packageMeinhart.functionsMasterProjectMeinhart as fmpm
import packageMeinhart.functionsFeaturesCrema as ffc


## Signal files

### Get all csv-files with signal data

In [2]:
# folder with csv-files
file_dir = 'E:\Physio_Data_Split_Ex_and_NonEx'

# get all file names of desired folder
signal_all_files = []
for (dirpath, dirnames, filenames) in os.walk(file_dir):
    signal_all_files.extend(filenames)
    break
print('Total number of files: {}'.format(len(signal_all_files)))
signal_all_files

Total number of files: 549


['subject01_00_nonEx.csv',
 'subject01_01_nonEx.csv',
 'subject01_02_nonEx.csv',
 'subject01_03_nonEx.csv',
 'subject01_04_nonEx.csv',
 'subject01_05_nonEx.csv',
 'subject01_06_nonEx.csv',
 'subject01_07_nonEx.csv',
 'subject01_08_nonEx.csv',
 'subject01_09_nonEx.csv',
 'subject01_10_nonEx.csv',
 'subject01_11_nonEx.csv',
 'subject01_12_nonEx.csv',
 'subject01_13_nonEx.csv',
 'subject01_14_nonEx.csv',
 'subject01_15_nonEx.csv',
 'subject01_16_nonEx.csv',
 'subject01_17_nonEx.csv',
 'subject01_18_nonEx.csv',
 'subject01_19_nonEx.csv',
 'subject01_20_nonEx.csv',
 'subject01_21_nonEx.csv',
 'subject01_22_nonEx.csv',
 'subject01_23_nonEx.csv',
 'subject01_24_nonEx.csv',
 'subject01_25_nonEx.csv',
 'subject01_26_nonEx.csv',
 'subject01_27_nonEx.csv',
 'subject01_28_nonEx.csv',
 'subject01_29_nonEx.csv',
 'subject01_30_nonEx.csv',
 'subject01_BC_05.csv',
 'subject01_BC_10.csv',
 'subject01_BC_15.csv',
 'subject01_LR_05.csv',
 'subject01_LR_10.csv',
 'subject01_LR_15.csv',
 'subject01_MP_05.c

### Remove files with exercise data

It is assumed that the features for the files with exercise data have already  been generated.

*(see Classification_according_to_Crema.ipynb)*

In [3]:
signal_files = [file for file in signal_all_files if 'nonEx' in file]
print('Number of exercise files: {}'.format(len(signal_files)))
signal_files

Number of exercise files: 279


['subject01_00_nonEx.csv',
 'subject01_01_nonEx.csv',
 'subject01_02_nonEx.csv',
 'subject01_03_nonEx.csv',
 'subject01_04_nonEx.csv',
 'subject01_05_nonEx.csv',
 'subject01_06_nonEx.csv',
 'subject01_07_nonEx.csv',
 'subject01_08_nonEx.csv',
 'subject01_09_nonEx.csv',
 'subject01_10_nonEx.csv',
 'subject01_11_nonEx.csv',
 'subject01_12_nonEx.csv',
 'subject01_13_nonEx.csv',
 'subject01_14_nonEx.csv',
 'subject01_15_nonEx.csv',
 'subject01_16_nonEx.csv',
 'subject01_17_nonEx.csv',
 'subject01_18_nonEx.csv',
 'subject01_19_nonEx.csv',
 'subject01_20_nonEx.csv',
 'subject01_21_nonEx.csv',
 'subject01_22_nonEx.csv',
 'subject01_23_nonEx.csv',
 'subject01_24_nonEx.csv',
 'subject01_25_nonEx.csv',
 'subject01_26_nonEx.csv',
 'subject01_27_nonEx.csv',
 'subject01_28_nonEx.csv',
 'subject01_29_nonEx.csv',
 'subject01_30_nonEx.csv',
 'subject02_00_nonEx.csv',
 'subject02_01_nonEx.csv',
 'subject02_02_nonEx.csv',
 'subject02_03_nonEx.csv',
 'subject02_04_nonEx.csv',
 'subject02_05_nonEx.csv',
 

## Feature generation

### Windowing all files with a window of 5 s and an increment of 0.2 s

In [4]:
# define csv-file to save the features
feature_save_dir = 'E:\Physio_Features_Crema'
feature_save_file = 'features_all_subjects_nonEx_5s_200ms.csv'
feature_save_path = os.path.join(feature_save_dir, feature_save_file)

# sampling rate of the signals
sampling_rate = 256

# window length
win_len = 5 # [s]

# window position increment
win_pos_inc = 0.2 # [s]

# parameters for the function fmpm.print_progress()
count = 0
max_count = len(signal_files)-1
prev_progress = 0 # previous progress

***Feature file for all subjects is only generated if it does not already exist. (takes ~25 min)***

In [5]:
# generate the feature file if it does not already exist
if not os.path.isfile(feature_save_path):

    # Generating the csv-file with the header
    ffc.write_crema_features_to_csv(feature_save_path, features=None)

    # going through all signal files
    for current_file in signal_files:

        # path to current csv-file with signals
        signal_path = os.path.join(file_dir, current_file)

        # load all signal data from current file
        signals = fmpm.get_sensor_data(in_file=signal_path, sampling_rate=sampling_rate, signals=['Acc','Gyr'])

        # window position at 0 s in the beginnig
        win_pos = 0 # [s]

        while True:

            # select signals of current window
            signals_5s = ffc.select_window(signals, window_length=win_len, start_time=win_pos, sampling_rate=sampling_rate)

            # break the while loop if signals_5s is None --> window out of range
            if signals_5s is None:
                break

            # perform the dimension reduction of the signals in the selected window
            sig_dim_red_5s = ffc.signal_dim_reduc(signals_5s)

            # generate the features from the dimension reduced signals
            features = ffc.generate_features_Crema(sig_dim_red_5s)

            # use the file name as label (without '.csv')
            label = current_file.split('.')[0]

            # write generated features to csv-file
            ffc.write_crema_features_to_csv(feature_save_path, features=features, label=label)

            # increase the window position
            win_pos += win_pos_inc

        #print the progress
        prev_progress = fmpm.print_progress(count, max_count, prev_progress)
        count += 1

### Load the generated features of non-exercise data

In [6]:
feature_data_nonEx = pd.read_csv(feature_save_path, sep=';')
feature_data_nonEx.head()

,label,ax_ampFreqDist_01,ax_ampFreqDist_02,ax_ampFreqDist_03,ax_ampFreqDist_04,ax_ampFreqDist_05,ax_rms_06,ax_mean_07,ax_std_08,ax_powerbands_09,...,gPC1_powerbands_15,gPC1_powerbands_16,gPC1_powerbands_17,gPC1_powerbands_18,gPC1_maxValAutocorr_19,gPC1_numPromPeaks_20,gPC1_numWeakPeaks_21,gPC1_valFirstPeak_22,gPC1_kurtosis_23,gPC1_intQuatRange_24
0,subject01_00_nonEx,2,2,2,366,908,0.015478,0.014756,0.004672,1.684952e-06,...,0.000336,0.000239,0.000375,0.000637,35.428112,1,0,0.000000,209.617445,0.069437
1,subject01_00_nonEx,6,66,876,278,54,0.015433,0.014994,0.003656,2.800491e-07,...,0.000403,0.000290,0.000442,0.000401,19.172494,1,1,4.565908,170.353124,0.079287
2,subject01_00_nonEx,5,68,875,279,53,0.015434,0.015000,0.003635,2.678981e-07,...,0.000114,0.000145,0.000102,0.000087,9.110770,0,10,0.684107,0.312396,0.092143
3,subject01_00_nonEx,68,279,593,286,54,0.015494,0.015070,0.003601,2.456812e-07,...,0.000111,0.000123,0.000110,0.000089,9.058118,0,10,0.606672,0.300821,0.086502
4,subject01_00_nonEx,66,278,591,289,56,0.015530,0.015106,0.003605,2.891163e-07,...,0.000122,0.000132,0.000153,0.000117,8.884359,0,8,0.499905,0.397488,0.092481


In [7]:
feature_data_nonEx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50363 entries, 0 to 50362
Data columns (total 73 columns):
label                       50363 non-null object
ax_ampFreqDist_01           50363 non-null int64
ax_ampFreqDist_02           50363 non-null int64
ax_ampFreqDist_03           50363 non-null int64
ax_ampFreqDist_04           50363 non-null int64
ax_ampFreqDist_05           50363 non-null int64
ax_rms_06                   50363 non-null float64
ax_mean_07                  50363 non-null float64
ax_std_08                   50363 non-null float64
ax_powerbands_09            50363 non-null float64
ax_powerbands_10            50363 non-null float64
ax_powerbands_11            50363 non-null float64
ax_powerbands_12            50363 non-null float64
ax_powerbands_13            50363 non-null float64
ax_powerbands_14            50363 non-null float64
ax_powerbands_15            50363 non-null float64
ax_powerbands_16            50363 non-null float64
ax_powerbands_17            50363 n

### Load also the features of exercise data

Not generated in this Notebook, must already exist.

*(see Classification_according_to_Crema.ipynb)*

In [8]:
feature_Ex_dir = 'E:\Physio_Features_Crema'
feature_Ex_file = 'features_all_subjects_5s_200ms.csv'
feature_Ex_path = os.path.join(feature_Ex_dir, feature_Ex_file)
feature_data_Ex = pd.read_csv(feature_Ex_path, sep=';')
feature_data_Ex.head()

,label,ax_ampFreqDist_01,ax_ampFreqDist_02,ax_ampFreqDist_03,ax_ampFreqDist_04,ax_ampFreqDist_05,ax_rms_06,ax_mean_07,ax_std_08,ax_powerbands_09,...,gPC1_powerbands_15,gPC1_powerbands_16,gPC1_powerbands_17,gPC1_powerbands_18,gPC1_maxValAutocorr_19,gPC1_numPromPeaks_20,gPC1_numWeakPeaks_21,gPC1_valFirstPeak_22,gPC1_kurtosis_23,gPC1_intQuatRange_24
0,subject01_BC_05,735,98,86,109,252,0.781195,-0.386669,0.678788,0.448270,...,0.248698,0.187060,0.114195,0.133722,1.564197e+07,1,0,5.677439e+06,-0.787818,159.182897
1,subject01_BC_05,684,98,86,110,302,0.774145,-0.324157,0.703009,0.483977,...,0.897272,0.723032,0.509787,0.429185,1.636813e+07,1,0,6.480917e+06,-0.923616,174.855254
2,subject01_BC_05,633,98,102,145,302,0.753763,-0.278716,0.700339,0.486481,...,1.782705,1.430826,1.007991,0.979056,1.789732e+07,1,0,7.681185e+06,-1.047846,185.516748
3,subject01_BC_05,584,127,122,145,302,0.733196,-0.254604,0.687571,0.471533,...,2.582355,2.178054,1.520687,1.318051,1.964916e+07,1,0,9.177501e+06,-1.209414,223.483180
4,subject01_BC_05,583,128,122,145,302,0.724777,-0.247308,0.681279,0.463399,...,1.668959,1.260825,0.938013,0.822285,2.080066e+07,1,0,1.016137e+07,-1.345619,250.921059


### Concatenate the two DataFrames with exercise and non-exercise data

In [9]:
feature_data_Ex_nonEx = pd.concat([feature_data_Ex, feature_data_nonEx], ignore_index=True)
feature_data_Ex_nonEx

,label,ax_ampFreqDist_01,ax_ampFreqDist_02,ax_ampFreqDist_03,ax_ampFreqDist_04,ax_ampFreqDist_05,ax_rms_06,ax_mean_07,ax_std_08,ax_powerbands_09,...,gPC1_powerbands_15,gPC1_powerbands_16,gPC1_powerbands_17,gPC1_powerbands_18,gPC1_maxValAutocorr_19,gPC1_numPromPeaks_20,gPC1_numWeakPeaks_21,gPC1_valFirstPeak_22,gPC1_kurtosis_23,gPC1_intQuatRange_24
0,subject01_BC_05,735,98,86,109,252,0.781195,-0.386669,0.678788,4.482702e-01,...,0.248698,0.187060,0.114195,0.133722,1.564197e+07,1,0,5.677439e+06,-0.787818,159.182897
1,subject01_BC_05,684,98,86,110,302,0.774145,-0.324157,0.703009,4.839773e-01,...,0.897272,0.723032,0.509787,0.429185,1.636813e+07,1,0,6.480917e+06,-0.923616,174.855254
2,subject01_BC_05,633,98,102,145,302,0.753763,-0.278716,0.700339,4.864809e-01,...,1.782705,1.430826,1.007991,0.979056,1.789732e+07,1,0,7.681185e+06,-1.047846,185.516748
3,subject01_BC_05,584,127,122,145,302,0.733196,-0.254604,0.687571,4.715332e-01,...,2.582355,2.178054,1.520687,1.318051,1.964916e+07,1,0,9.177501e+06,-1.209414,223.483180
4,subject01_BC_05,583,128,122,145,302,0.724777,-0.247308,0.681279,4.633993e-01,...,1.668959,1.260825,0.938013,0.822285,2.080066e+07,1,0,1.016137e+07,-1.345619,250.921059
5,subject01_BC_05,591,123,123,142,301,0.724313,-0.246951,0.680914,4.629183e-01,...,0.310056,0.309890,0.189937,0.163268,2.142876e+07,1,0,1.055472e+07,-1.424096,250.943892
6,subject01_BC_05,591,123,123,142,301,0.724196,-0.246859,0.680823,4.627805e-01,...,0.034838,0.014833,0.010470,0.006427,2.147039e+07,1,0,1.056765e+07,-1.430062,250.948389
7,subject01_BC_05,591,123,123,142,301,0.725181,-0.247631,0.681591,4.638392e-01,...,0.064066,0.045407,0.031422,0.026504,2.139034e+07,1,0,1.034049e+07,-1.421949,250.933471
8,subject01_BC_05,591,123,123,142,301,0.726586,-0.248883,0.682631,4.652916e-01,...,0.229462,0.167779,0.119116,0.112185,2.078664e+07,1,1,9.903861e+06,-1.420477,247.117479
9,subject01_BC_05,604,122,111,142,301,0.731554,-0.256373,0.685159,4.682699e-01,...,0.065229,0.035210,0.028574,0.022295,2.000815e+07,1,1,9.549079e+06,-1.455245,247.057473


## Defining the test subject $\rightarrow$ split training and test data

For training of the ML model data from all subjects are used except those from the test subject.

In [10]:
# defining the test subject
test_subject = 'subject01'

# exercises to evaluate --> abbreviations
ex_abbr = ['RF', 'RO', 'RS', 'LR', 'BC', 'TC', 'MP', 'SA', 'P1', 'P2', 'NE']
    
# repetition blocks to evaluate
rep_block_nums = ['05', '10', '15']

# exercise dictionary
label_ex = {}
for ii, ex in enumerate(ex_abbr):
    label_ex[ex] = ii

### Training data

In [11]:
# generate the label array (as well as a list with the label indices)
y_train = []
ind_train = []
for ii, label in enumerate(feature_data_Ex_nonEx.values[:,0]):
    if test_subject not in label:
        if 'nonEx' in label:
            ind_train.append(ii)
            y_train.append(label_ex['NE'])
        else:
            ind_train.append(ii)
            y_train.append(label_ex[re.split('[_]', label)[1]])

# get feature matrix for training
X_train = feature_data_Ex_nonEx.values[ind_train,1:]

print('Size of feature matrix for training: {} x {}'.format(np.shape(X_train)[0], np.shape(X_train)[1]))
print('Length of corresponding label array: {}'.format(len(y_train)))

Size of feature matrix for training: 77684 x 72
Length of corresponding label array: 77684


### Test data

This is now a little bit tricky because we want to have the features of the whole record of the test subject.
This means that the sliding window shall also cover the passages from exercises to non-exercises and vice versa.
Hence, it is necessary to window the test subject data again as a whole.

In [12]:
# define csv-file to save the test subject features
feature_test_subject_save_dir = 'E:\Physio_Features_Crema'
feature_test_subject_save_file = 'features_' + test_subject + '_5s_200ms.csv'
feature_test_subject_save_path = os.path.join(feature_test_subject_save_dir, feature_test_subject_save_file)

# sampling rate of the signals
sampling_rate = 256

# window length
win_len = 5 # [s]

# window position increment
win_pos_inc = 0.2 # [s]

# path to the csv-file with the whole record of the test subject
test_subject_dir = 'E:\Physio_Data\Subject_' + test_subject[-2:] # last two characters of the test subject (e.g. '01')
test_subject_file = test_subject + '.csv'
test_subject_path = os.path.join(test_subject_dir, test_subject_file)

***Feature file for the test subject is only generated if it does not already exist.***

In [13]:
# generate the feature file if it does not already exist
if not os.path.isfile(feature_test_subject_save_path):
    
    # Generating the csv-file with the header
    ffc.write_crema_features_to_csv(feature_test_subject_save_path, features=None)
    
    # load all signal data from test subject file
    signals = fmpm.get_sensor_data(in_file=test_subject_path, sampling_rate=sampling_rate, signals=['Acc','Gyr'])

    # parameters for the function fmpm.print_progress()
    count = 0
    max_count = (signals['time'][-1] - win_len) / win_pos_inc
    prev_progress = 0 # previous progress

    # window position at 0 s in the beginnig
    win_pos = 0 # [s]

    while True:

        # select signals of current window
        signals_5s = ffc.select_window(signals, window_length=win_len, start_time=win_pos, sampling_rate=sampling_rate)

        # break the while loop if signals_5s is None --> window out of range
        if signals_5s is None:
            break

        # perform the dimension reduction of the signals in the selected window
        sig_dim_red_5s = ffc.signal_dim_reduc(signals_5s)

        # generate the features from the dimension reduced signals
        features = ffc.generate_features_Crema(sig_dim_red_5s)

        # write generated features to csv-file
        ffc.write_crema_features_to_csv(feature_test_subject_save_path, features=features, label='none')

        # increase the window position
        win_pos += win_pos_inc

        #print the progress
        prev_progress = fmpm.print_progress(count, max_count, prev_progress)
        count += 1
        
fmpm.print_progress(max_count, max_count, prev_progress);

Progress: 100%


***Load the features of the test subject.***

In [14]:
feature_data_test_subject = pd.read_csv(feature_test_subject_save_path, sep=';')
feature_data_test_subject

,label,ax_ampFreqDist_01,ax_ampFreqDist_02,ax_ampFreqDist_03,ax_ampFreqDist_04,ax_ampFreqDist_05,ax_rms_06,ax_mean_07,ax_std_08,ax_powerbands_09,...,gPC1_powerbands_15,gPC1_powerbands_16,gPC1_powerbands_17,gPC1_powerbands_18,gPC1_maxValAutocorr_19,gPC1_numPromPeaks_20,gPC1_numWeakPeaks_21,gPC1_valFirstPeak_22,gPC1_kurtosis_23,gPC1_intQuatRange_24
0,none,2,2,2,366,908,0.015478,0.014756,0.004672,1.684952e-06,...,0.000336,0.000239,0.000375,0.000637,35.428112,1,0,0.000000,209.617445,0.069437
1,none,6,66,876,278,54,0.015433,0.014994,0.003656,2.800491e-07,...,0.000403,0.000290,0.000442,0.000401,19.172494,1,1,4.565908,170.353124,0.079287
2,none,5,68,875,279,53,0.015434,0.015000,0.003635,2.678981e-07,...,0.000114,0.000145,0.000102,0.000087,9.110770,0,10,0.684107,0.312396,0.092143
3,none,68,279,593,286,54,0.015494,0.015070,0.003601,2.456812e-07,...,0.000111,0.000123,0.000110,0.000089,9.058118,0,10,0.606672,0.300821,0.086502
4,none,66,278,591,289,56,0.015530,0.015106,0.003605,2.891163e-07,...,0.000122,0.000132,0.000153,0.000117,8.884359,0,8,0.499905,0.397488,0.092481
5,none,64,273,594,297,52,0.015549,0.015134,0.003570,2.925819e-07,...,0.000123,0.000125,0.000131,0.000112,8.830903,0,8,0.566894,0.336805,0.093416
6,none,62,274,595,296,53,0.015560,0.015149,0.003555,2.943664e-07,...,0.000137,0.000142,0.000153,0.000105,9.029366,0,8,0.603004,0.396932,0.089532
7,none,65,281,587,291,56,0.015535,0.015112,0.003598,3.355274e-07,...,0.000137,0.000147,0.000126,0.000091,8.906056,0,8,0.609449,0.364413,0.089113
8,none,56,286,587,296,55,0.015571,0.015161,0.003548,2.762233e-07,...,0.000126,0.000130,0.000152,0.000089,8.801027,0,6,0.498580,0.357764,0.100673
9,none,56,288,587,292,57,0.015567,0.015155,0.003558,2.771272e-07,...,0.000139,0.000126,0.000120,0.000080,8.653364,0,8,0.578250,0.417057,0.091819


In [15]:
# omit first column for feature matrix (labels)
X_test = feature_data_test_subject.values[:,1:]

## ML model

In [16]:
# create random forest classifier model
ML_model = RandomForestClassifier(n_estimators=250, max_leaf_nodes=30, n_jobs=-1, random_state=42)

### Train the model

In [17]:
ML_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=30,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

### Classify the test data

In [18]:
y_pred = ML_model.predict(X_test) # classify
y_probs = ML_model.predict_proba(X_test) # predict probabilities

In [19]:
# One Hot Encoding --> DataFrame for plotting afterwards

# generate matrix and reserve memory
y_pred_OHE = np.zeros((len(y_pred),len(ex_abbr)))

# going through all exercises
for ex in ex_abbr:
    # adding the array with 0/1 values to corresponding columns
    y_pred_OHE[:,label_ex[ex]] = (y_pred==label_ex[ex]).astype(int)
    
# creating the dataFrame
y_pred_df = pd.DataFrame(y_pred_OHE, columns=ex_abbr)
y_pred_df.head()

,RF,RO,RS,LR,BC,TC,MP,SA,P1,P2,NE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Comparing predicted with actual classes

### First we have to load the actual time ranges of the exercises of the test data

In [20]:
# loading the actual time ranges of the exercises of the test data for the subsequent plot 
# to compare with predicted values

# select file with timetable (csv) of the test subject
timetable_file_dir = r'E:\Physio_Data\Exercise_time_tables'
timetable_file_name = 'Timetable_' + test_subject + '.txt'
timetable_data_path = os.path.join(timetable_file_dir, timetable_file_name)

# read in time table
timetable_data = pd.read_csv(timetable_data_path, skiprows=0, sep='\t', header=None)
num_exercises = timetable_data.shape[0] # number of exercises
timetable_data

,0,1,2,3,4,5,6,7,8,9
0,Raises Oblique,15,5,10,01:18.6,01:58.3,02:22.1,02:37.1,02:54.8,03:23.3
1,PNF Diagonal 2,10,5,15,04:27.1,04:54.3,05:24.5,05:38.9,06:25.8,07:05.1
2,Triceps Curls,15,5,10,07:32.3,08:14.8,08:49.5,09:04.9,09:46.1,10:12.6
3,Rotation Wrist,5,10,15,10:43.1,10:57.3,11:25.6,11:51.8,12:12.1,12:52.4
4,Raises Front,10,15,5,13:44.8,14:14,14:40.6,15:20,16:09.5,16:23.1
5,Biceps Curls,15,10,5,16:50.8,17:30.2,18:01.3,18:32.2,18:58.6,19:12
6,Raises Side,15,5,10,20:13,21:00.2,21:24.9,21:40.2,22:09.6,22:38.4
7,PNF Diagonal 1,10,15,5,23:43.9,24:15.9,24:42,25:25.1,25:48.7,26:04.6
8,Shoulder Adduct.,5,15,10,26:48.5,27:05.3,27:25.4,28:16.5,28:41.2,29:10.9
9,Military Press,15,10,5,29:45.6,30:37.4,31:14.6,31:47.3,32:13.6,32:28.1


### Plotting of predicted and actual classes

In [ ]:
%matplotlib auto

# number of data points
num_data_points = len(y_pred)

# text for current subject
sub_text = 'Subject {}\n'.format(int(test_subject.split('t')[1])) # Subject number

fig, axis = plt.subplots(12,1,figsize=(18,9), sharex=True)

# image color settings for RFC probabilities
cmap = plt.cm.seismic
vmin=0
vmax=1

for ax, ex in zip(axis, ex_abbr):
    #y_pred_df[ex].plot(kind='bar', width = 1)
    ax.fill_between(range(num_data_points), y_pred_df[ex].values, color='b')
    ax.plot(y_probs[:,label_ex[ex]], 'r')
    ax.set_ylim([0,1])
    ax.set_yticks([0,0.5,1])
    ax.set_yticklabels([0,50,100], fontsize=7)
    ax.set_ylabel(ex, rotation=0, fontsize=13)
    ax.yaxis.labelpad = 32
    ax.xaxis.set_ticklabels([])
    
plt.gcf().text(0.1, 0.7, 'predicted class and probability [%]', fontsize=10, rotation=90)

axis[-1].plot(range(num_data_points), np.zeros(num_data_points), 'k', alpha=0.0)
formatter = matplotlib.ticker.FuncFormatter(lambda i, x: time.strftime('%M:%S', time.gmtime(i*win_pos_inc)))
axis[-1].xaxis.set_major_formatter(formatter)
axis[-1].set_xlabel(r'time $[min:sec]$', fontsize=13)
axis[-1].set_yticks([])
axis[-1].set_ylim([0,1])

fig.subplots_adjust(bottom=0.2) # make space for buttons

# add slider for selections on the x axis
Slider_shiftX_ax = plt.axes([0.125, 0.07, 0.775, 0.025])
Slider_zoomX_ax = plt.axes([0.125, 0.035, 0.775, 0.025])

axcolor = 'cornflowerblue'
Slider_shiftX = Slider(Slider_shiftX_ax, 'time shift [%]', 0.0, 100.0, valinit=0, facecolor=axcolor)
Slider_zoomX = Slider(Slider_zoomX_ax, 'time scale [%]', 0.1, 100.0, valinit=100, facecolor=axcolor)
Slider_zoomX_ax.xaxis.set_visible(True)
Slider_zoomX_ax.set_xticks(np.arange(0,105,5))

# function to convert indices to time strings
def indices_to_time(start_index, stop_index):
    start_time_text = '{0:02}:{1:04.1f}'.format(int(start_index*win_pos_inc/60), 
                                               (start_index*win_pos_inc)%60)
    stop_time_text = '{0:02}:{1:04.1f}'.format(int(stop_index*win_pos_inc/60), 
                                               (stop_index*win_pos_inc)%60)
    return start_time_text + ' - ' + stop_time_text

def updateX(val):
    start_index = int(Slider_shiftX.val / 100 * num_data_points)
    stop_index = start_index + Slider_zoomX.val / 100 * num_data_points
    axis[-1].set_xlim((start_index, stop_index))
    fig.suptitle('Predicted Probabilities ' + sub_text + indices_to_time(start_index, stop_index), fontsize=20)
    plt.draw()

Slider_shiftX.on_changed(updateX)
Slider_zoomX.on_changed(updateX)

# add button to reset view
def resetX(val):
    start_index = 0
    stop_index = num_data_points
    axis[-1].set_xlim((start_index, stop_index))
    Slider_shiftX.reset()
    Slider_zoomX.reset()
    fig.suptitle('Predicted Probabilities ' + sub_text + indices_to_time(start_index, stop_index), fontsize=20)
    plt.draw()

Button_resetX_ax = plt.axes([0.85, 0.12, 0.05, 0.03])
Button_resetX = Button(Button_resetX_ax, 'Reset view')
Button_resetX.on_clicked(resetX)

start_index = 0
stop_index = num_data_points

fig.suptitle('Predicted Probabilities ' + sub_text + indices_to_time(start_index, stop_index), fontsize=20)

axis[-1].set_xlim(0, num_data_points)


# Plotting the actual classes (exercises) on the last axis:

# dictionary to get exercise abbreviations from exercise names in timetable
exercise_names = {'Raises Front':'RF',
                  'Raises Oblique':'RO',
                  'Raises Side':'RS',
                  'Rotation Wrist':'LR',
                  'Biceps Curls':'BC',
                  'Triceps Curls':'TC',
                  'Military Press':'MP',
                  'Shoulder Adduct.':'SA',
                  'PNF Diagonal 1':'P1',
                  'PNF Diagonal 2':'P2'}

# going through all exercises in the timetable
for ii, ex_name in enumerate(timetable_data.values[:,0]):
    
    # going through all repetition blocks in the timetable (5, 10 and 15 rep. blocks)
    for rep_col, start_col, stop_col in zip([1,2,3],[4,6,8],[5,7,9]): # corresponding columns
        rep_num = timetable_data.values[ii,rep_col]
        left_border = fmpm.convert_time_format_to_index(timetable_data.values[ii,start_col], 
                                                        sampling_rate=1/win_pos_inc)
        right_border = fmpm.convert_time_format_to_index(timetable_data.values[ii,stop_col], 
                                                         sampling_rate=1/win_pos_inc)
        # mark the corresponding area
        axis[-1].axvspan(left_border, right_border, color='y', alpha=0.3, lw=0)
        # write text to the corresponding area
        x_center = left_border + (right_border-left_border)/2 # x center of marked area
        axis[-1].text(x_center, 0.5, str(rep_num) + '\n' + exercise_names[ex_name], 
                      horizontalalignment='center', verticalalignment='center', fontsize=10, clip_on=True)
        
axis[-1].set_ylabel('Actual classes', rotation=0, fontsize=11)
axis[-1].yaxis.labelpad = 50

plt.show()

In [28]:
%matplotlib auto

# number of data points
num_data_points = len(y_pred)

# text for current subject
sub_text = 'Subject {}\n'.format(int(test_subject.split('t')[1])) # Subject number

fig, axis = plt.subplots(12,1,figsize=(15,8), sharex=True)

# image color settings for RFC probabilities
cmap = plt.cm.seismic
vmin=0
vmax=1

for ax, ex in zip(axis, ex_abbr):
    #y_pred_df[ex].plot(kind='bar', width = 1)
    ax.fill_between(range(num_data_points), y_pred_df[ex].values, color='b', linewidth=0.01)
    ax.plot(y_probs[:,label_ex[ex]], 'r', linewidth=1)
    ax.set_ylim([0,1])
    ax.set_yticks([0,0.5,1])
    ax.set_yticklabels([0,50,100], fontsize=10)
    ax.set_ylabel(ex, rotation=0, fontsize=16)
    ax.yaxis.labelpad = 50
    ax.xaxis.set_ticklabels([])
    
plt.gcf().text(0.1, 0.7, 'predicted class and probability [%]', fontsize=14, rotation=90)

axis[-1].plot(range(num_data_points), np.zeros(num_data_points), 'k', alpha=0.0)
formatter = matplotlib.ticker.FuncFormatter(lambda i, x: time.strftime('%M:%S', time.gmtime(i*win_pos_inc)))
axis[-1].xaxis.set_major_formatter(formatter)
axis[-1].xaxis.set_tick_params(labelsize=11)
axis[-1].set_xlabel(r'time $[min:sec]$', fontsize=14)
axis[-1].set_yticks([])
axis[-1].set_ylim([0,1])



start_index = 0
stop_index = num_data_points

fig.suptitle('Predicted Probabilities ' + sub_text, fontsize=20)

axis[-1].set_xlim(0, num_data_points)


# Plotting the actual classes (exercises) on the last axis:

# dictionary to get exercise abbreviations from exercise names in timetable
exercise_names = {'Raises Front':'RF',
                  'Raises Oblique':'RO',
                  'Raises Side':'RS',
                  'Rotation Wrist':'LR',
                  'Biceps Curls':'BC',
                  'Triceps Curls':'TC',
                  'Military Press':'MP',
                  'Shoulder Adduct.':'SA',
                  'PNF Diagonal 1':'P1',
                  'PNF Diagonal 2':'P2'}

# going through all exercises in the timetable
for ii, ex_name in enumerate(timetable_data.values[:,0]):
    
    # going through all repetition blocks in the timetable (5, 10 and 15 rep. blocks)
    for rep_col, start_col, stop_col in zip([1,2,3],[4,6,8],[5,7,9]): # corresponding columns
        rep_num = timetable_data.values[ii,rep_col]
        left_border = fmpm.convert_time_format_to_index(timetable_data.values[ii,start_col], 
                                                        sampling_rate=1/win_pos_inc)
        right_border = fmpm.convert_time_format_to_index(timetable_data.values[ii,stop_col], 
                                                         sampling_rate=1/win_pos_inc)
        # mark the corresponding area
        axis[-1].axvspan(left_border, right_border, color='y', alpha=0.3, lw=0)
        # write text to the corresponding area
        x_center = left_border + (right_border-left_border)/2 # x center of marked area
        axis[-1].text(x_center, 0.5, str(rep_num) + '\n' + exercise_names[ex_name], 
                      horizontalalignment='center', verticalalignment='center', fontsize=11, clip_on=True)
        
axis[-1].set_ylabel('Actual classes', rotation=0, fontsize=12)
axis[-1].yaxis.labelpad = 50

# change layout for saving
#plt.tight_layout(rect=(0.015, 0, 0.9, 0.9), pad=0.2)
plt.tight_layout(rect=(0.057, 0, 0.9, 0.93), pad=0.2)

plt.show()

fig_save_name = 'Predicted_Probabilities_' + test_subject + '.png'

fig_save_dir = r'C:\Users\Florian_Meinhart\Desktop\Plots_1st_Approach'
fig_save_path = os.path.join(fig_save_dir, fig_save_name)

# Save the figure if it does not already exist:
if not os.path.exists(fig_save_path):
    plt.savefig(fig_save_path, dpi=200)
    print('Figure saved.')
else:
    print('Figure already exists.')

Using matplotlib backend: TkAgg
Figure already exists.


In [35]:
%matplotlib auto

# number of data points
num_data_points = len(y_pred)

# text for current subject
sub_text = 'Subject {}\n'.format(int(test_subject.split('t')[1])) # Subject number

fig, axis = plt.subplots(12,1,figsize=(17,8), sharex=True)

# image color settings for RFC probabilities
cmap = plt.cm.seismic
vmin=0
vmax=1

for ax, ex in zip(axis, ex_abbr):
    #y_pred_df[ex].plot(kind='bar', width = 1)
    ax.fill_between(range(num_data_points), y_pred_df[ex].values, color='b', linewidth=0.01)
    ax.plot(y_probs[:,label_ex[ex]], 'r', linewidth=1)
    ax.set_ylim([0,1])
    ax.set_yticks([0,0.5,1])
    ax.set_yticklabels([0,50,100], fontsize=10)
    ax.set_ylabel(ex, rotation=0, fontsize=16)
    ax.yaxis.labelpad = 50
    ax.xaxis.set_ticklabels([])
    
plt.gcf().text(0.1, 0.7, 'predicted class and probability [%]', fontsize=14, rotation=90)

axis[-1].plot(range(num_data_points), np.zeros(num_data_points), 'k', alpha=0.0)
formatter = matplotlib.ticker.FuncFormatter(lambda i, x: time.strftime('%M:%S', time.gmtime(i*win_pos_inc)))
axis[-1].xaxis.set_major_formatter(formatter)
axis[-1].xaxis.set_tick_params(labelsize=11)
axis[-1].set_xlabel(r'time $[min:sec]$', fontsize=14)
axis[-1].set_yticks([])
axis[-1].set_ylim([0,1])



start_index = 0
stop_index = num_data_points

fig.suptitle('Predicted Probabilities ' + sub_text, fontsize=20)

axis[-1].set_xlim(0, num_data_points)


# Plotting the actual classes (exercises) on the last axis:

# dictionary to get exercise abbreviations from exercise names in timetable
exercise_names = {'Raises Front':'RF',
                  'Raises Oblique':'RO',
                  'Raises Side':'RS',
                  'Rotation Wrist':'LR',
                  'Biceps Curls':'BC',
                  'Triceps Curls':'TC',
                  'Military Press':'MP',
                  'Shoulder Adduct.':'SA',
                  'PNF Diagonal 1':'P1',
                  'PNF Diagonal 2':'P2'}

# going through all exercises in the timetable
for ii, ex_name in enumerate(timetable_data.values[:,0]):
    
    # going through all repetition blocks in the timetable (5, 10 and 15 rep. blocks)
    for rep_col, start_col, stop_col in zip([1,2,3],[4,6,8],[5,7,9]): # corresponding columns
        rep_num = timetable_data.values[ii,rep_col]
        left_border = fmpm.convert_time_format_to_index(timetable_data.values[ii,start_col], 
                                                        sampling_rate=1/win_pos_inc)
        right_border = fmpm.convert_time_format_to_index(timetable_data.values[ii,stop_col], 
                                                         sampling_rate=1/win_pos_inc)
        # mark the corresponding area
        axis[-1].axvspan(left_border, right_border, color='y', alpha=0.3, lw=0)
        # write text to the corresponding area
        x_center = left_border + (right_border-left_border)/2 # x center of marked area
        axis[-1].text(x_center, 0.5, str(rep_num) + '\n' + exercise_names[ex_name], 
                      horizontalalignment='center', verticalalignment='center', fontsize=11, clip_on=True)
        
axis[-1].set_ylabel('Actual classes', rotation=0, fontsize=12)
axis[-1].yaxis.labelpad = 50

# change layout for saving
#plt.tight_layout(rect=(0.015, 0, 0.9, 0.9), pad=0.2)
plt.tight_layout(rect=(0.06, 0, 0.9, 0.93), pad=0.2)

plt.show()

fig_save_name = 'Predicted_Probabilities_' + test_subject + '_new.png'

fig_save_dir = r'C:\Users\Florian_Meinhart\Desktop\Plots_1st_Approach'
fig_save_path = os.path.join(fig_save_dir, fig_save_name)

# Save the figure if it does not already exist:
if not os.path.exists(fig_save_path):
    plt.savefig(fig_save_path, dpi=200)
    print('Figure saved.')
else:
    print('Figure already exists.')

Using matplotlib backend: TkAgg
Figure already exists.
